In [1]:
import pandas as pd

In [2]:
jc = pd.read_csv('file.txt', sep='|', dtype='str')

In [3]:
#jc,

In [4]:
jc.columns

Index(['IS_INTERNAL_YN', 'INTERNAL_DEBIT', 'INTERNAL_CREDIT', 'FIRST',
       'FIRST_DEBIT', 'FIRST_CREDIT', 'SECOND', 'SECOND_DEBIT',
       'SECOND_CREDIT', 'THIRD', 'THIRD_DEBIT', 'THIRD_CREDIT', 'EXP_DATE',
       'RECEIPT_NO', 'GUEST_FULL_NAME', 'TARGET_RESORT', 'TRX_DESC',
       'MARKET_CODE', 'BUSINESS_FORMAT_DATE', 'BUSINESS_TIME', 'BUSINESS_DATE',
       'REFERENCE', 'TRX_NO', 'CASHIER_DEBIT', 'CASHIER_CREDIT', 'ROOM',
       'CREDIT_CARD_SUPPLEMENT', 'CURRENCY1', 'TRX_CODE', 'CASHIER_ID',
       'REMARK', 'INSERT_USER', 'INSERT_DATE', 'CHEQUE_NUMBER', 'ROOM_CLASS',
       'CC_CODE', 'CASHIER_NAME', 'USER_NAME', 'DEP_NET_TAX_AMT',
       'DEPOSIT_DEBIT', 'CASH_ID_USER_NAME', 'PRINT_CASHIER_DEBIT',
       'PRINT_CASHIER_CREDIT'],
      dtype='object')

In [5]:
rest_cols = [
    'THIRD_DEBIT', 'THIRD_CREDIT', 'EXP_DATE',
    'RECEIPT_NO', 'GUEST_FULL_NAME', 'TARGET_RESORT', 'TRX_DESC',
    'MARKET_CODE', 'BUSINESS_FORMAT_DATE', 'BUSINESS_TIME', 'BUSINESS_DATE',
    'REFERENCE', 'TRX_NO', 'CASHIER_DEBIT', 'CASHIER_CREDIT', 'ROOM',
    'CREDIT_CARD_SUPPLEMENT', 'CURRENCY1', 'TRX_CODE', 'CASHIER_ID',
    'REMARK', 'INSERT_USER', 'INSERT_DATE'
    ]

In [6]:
#jc[rest_cols],

In [7]:
jc.isna().any()

IS_INTERNAL_YN            False
INTERNAL_DEBIT            False
INTERNAL_CREDIT            True
FIRST                      True
FIRST_DEBIT                True
FIRST_CREDIT               True
SECOND                     True
SECOND_DEBIT               True
SECOND_CREDIT              True
THIRD                      True
THIRD_DEBIT                True
THIRD_CREDIT               True
EXP_DATE                   True
RECEIPT_NO                 True
GUEST_FULL_NAME            True
TARGET_RESORT              True
TRX_DESC                   True
MARKET_CODE                True
BUSINESS_FORMAT_DATE       True
BUSINESS_TIME              True
BUSINESS_DATE              True
REFERENCE                  True
TRX_NO                     True
CASHIER_DEBIT              True
CASHIER_CREDIT             True
ROOM                       True
CREDIT_CARD_SUPPLEMENT     True
CURRENCY1                  True
TRX_CODE                   True
CASHIER_ID                 True
REMARK                     True
INSERT_U

In [8]:
# after seeing that last two rows are all NaN, we can get rid of them
jc.drop(jc.tail(2).index, inplace=True)
jc.fillna('', inplace=True)

In [9]:
jc.shape

(52, 43)

In [10]:
needed_columns = ['TRX_CODE', 'BUSINESS_TIME', 'GUEST_FULL_NAME', 'REFERENCE',
                  'CREDIT_CARD_SUPPLEMENT', 'ROOM', 'CURRENCY1',
                  'CASHIER_CREDIT', 'CASH_ID_USER_NAME']

In [11]:
jc = jc[needed_columns]

In [12]:
jc.columns = [column.lower() for column in jc.columns]

In [13]:
convenient_col_names_dict = {'business_time': 'trx_time',
                             'guest_full_name': 'guest_name',
                             'reference': 'note1',
                             'credit_card_supplement': 'note2',
                             'currency1': 'currency',
                             'cashier_credit': 'trx_amount',
                             'cash_id_user_name': 'username'
                            }

In [14]:
jc.rename(columns=convenient_col_names_dict, inplace=True)

In [15]:
jc.columns

Index(['trx_code', 'trx_time', 'guest_name', 'note1', 'note2', 'room',
       'currency', 'trx_amount', 'username'],
      dtype='object')

In [16]:
#jc,

In [17]:
# working with text of columns
columns = ['note1', 'note2', 'guest_name']
for column in columns[:2]:
    # make all text lowercase
    jc[column] = jc[column].apply(lambda x: x.lower() if isinstance(x, str) else x)
    # replace all POS(payment on spot) notes
    jc[column] = jc[column].replace(r'^check# 0([0-9]{6}) \[[0-9]{5}\]', r'pos \1', regex=True)
    # replace all text containing bank card numbers
    jc[column] = jc[column].replace(r'x+([0-9]{4})|(x{3,20}[a-z]*\.)', r'xx\1', regex=True)
    jc[column] = jc[column].replace(r'[\.]+([0-9]{4})', r'xx\1', regex=True)
    jc[column] = jc[column].replace(r'(xx[0-9]{4})( xx[0-9]{4}| xx$)', r'\1', regex=True)
    jc[column] = jc[column].replace(r'[ ]{2,9}([0-9]{1,2}\/[0-9]{1,2})', r' \1', regex=True)

    if column == columns[0]:
        jc[column] = jc[column].replace(r'([0-9]{1,3}),([0-9]+\.[0-9]{2,}) usd split into ', r'[\1\2=',
                                        regex=True)
        jc[column] = jc[column].replace(
            r'([0-9]{1,3}),([0-9]+\.[0-9]{2,}) usd and ([0-9]{1,3}),([0-9]+\.[0-9]{2,}) ', r'\1\2+\3\4',
            regex=True)
        jc[column] = jc[column].replace(r'([0-9]+)usd. ', r'\1]', regex=True)
        jc[column] = jc[column].replace(r'(\[[0-9.=+]+\])[a-z ]+([0-9.=>#]+)[a-z ]+([0-9#]+)', r'\1 \2\3',
                                        regex=True)
        jc[column] = jc[column].replace(r'[ ]+', ' ', regex=True)
    else:
        jc[column] = jc[column].replace(r'[ ]+', ' ', regex=True)


In [18]:
# removing sensitive info
jc.drop([3, 4, 6], axis='rows', inplace=True)

In [19]:
jc[columns[:2]],

(                                                note1         note2
 0                                              xx7269        xx7269
 1                                        cc type corr  cc type corr
 2                                              xx4121        xx4121
 5                                              xx7269        xx7269
 7                                          pos 049740    pos 049740
 8                                          pos 049783    pos 049783
 9                                          pos 049789    pos 049789
 10                                         pos 049800    pos 049800
 11                                         pos 049843    pos 049843
 12                                         pos 049849    pos 049849
 13                                         pos 049738    pos 049738
 14                                         pos 049870    pos 049870
 15                                         pos 049877    pos 049877
 16                               

In [20]:
jc['guest_name']

0                         Cohen,Hadas
1                         Cohen,Hadas
2                         Peretz,Erez
5                         Cohen,Hadas
7                       POS VISA,9191
8                       POS VISA,9191
9                       POS VISA,9191
10                      POS VISA,9191
11                      POS VISA,9191
12                      POS VISA,9191
13                      POS VISA,9191
14                      POS VISA,9191
15                      POS VISA,9191
16                      POS VISA,9191
17                      POS VISA,9191
18                      POS VISA,9191
19                      POS VISA,9191
20                POS MASTERCARD,9194
21                POS MASTERCARD,9194
22                POS MASTERCARD,9194
23                POS MASTERCARD,9194
24                POS MASTERCARD,9194
25                POS MASTERCARD,9194
26                POS MASTERCARD,9194
27                POS MASTERCARD,9194
28                POS MASTERCARD,9194
29          

In [21]:
column = columns[2]

In [22]:
jc[column] = jc[column].replace(r'[ ,]*(And|and)[ ,]*', r' + ', regex=True)

In [23]:
jc[column] = jc[column].replace(r'^(Mrs|mrs|Mr|mr)|(Mrs|mrs|Mr|mr)$|[ ,]*(Mrs|mrs|Mr|mr)|(Mrs|mrs|Mr|mr)[ ,]*$', '', regex=True)

In [24]:
jc[column] = jc[column].replace(r'[ +]+$', '', regex=True)

In [25]:
# trim whitespaces in note1, note2, guest_name columns
for column in columns:
    jc[column] = jc[column].apply(lambda x: x.strip())


In [26]:
# add new all in one notes column
notes = []
for index, row in jc.iterrows():
    if row['note1'] == "" and row['note2'] == "":
        notes.append("empty")
    elif row['note1'] == "":
        notes.append(row['note2'])
    elif row['note2'] == "":
        notes.append(row['note1'])
    elif row['note1'] != row['note2']:
        notes.append(f"{row['note2']} :: {row['note1']}")
    else:
        notes.append(row['note1'])


In [27]:
print(notes)

['xx7269', 'cc type corr', 'xx4121', 'xx7269', 'pos 049740', 'pos 049783', 'pos 049789', 'pos 049800', 'pos 049843', 'pos 049849', 'pos 049738', 'pos 049870', 'pos 049877', 'pos 049885', 'pos 023450', 'pos 023489', 'pos 023490', 'pos 023418', 'pos 049811', 'pos 049846', 'pos 023430', 'pos 049864', 'pos 023432', 'pos 049868', 'pos 023444', 'pos 023441', 'pos 023457', 'xx9303', 'xx1007', 'xx1007', 'xx1249', 'xx4942', 'xx4985', 'xx0354', 'xx2853 :: [5620.76=3099.59+2521.17]', 'xx2853 :: [5620.76=3099.59+2521.17] #1712=>#1724', 'xx2853 :: [6186.02=3008.74+3177.28]', 'xx2853 :: [6186.02=3008.74+3177.28] #1722=>#1711 [3177.28=1486.38+1690.90]', 'xx2853 :: [6186.02=3008.74+3177.28] #1722=>#1711 [3177.28=1486.38+1690.90] #1711=>#1709', 'xx1985', 'xx1985', 'xx9099', 'xx5150', 'xx5549', 'xx4444', 'xx9315', 'xx0419', 'xx0419', 'xx4121']


In [28]:
jc['note'] = notes

In [29]:
# convert column datatype to float for future calculations
jc['trx_amount'] = jc['trx_amount'].astype(float)

In [30]:
# as an example we take rate of 3.75 for 1 USD
rate = 3.75

In [31]:
jc['ils'] = jc['trx_amount'].apply(lambda x: round(x * rate, 2))

In [32]:
final_columns = ['username', 'trx_code', 'trx_time', 'guest_name',
                 'note', 'room', 'currency', 'ils',
                 'trx_amount']

In [33]:
jc = jc[final_columns]